In [1]:
import numpy as np
import pandas as pd

import ast
import os

In [2]:
def parse_results(file_name):
    grid_search_results = []
    with open(file_name, 'r') as file:
        for line in file.readlines():
            if line[0:4] == '----':
                grid_search_results.append([])
                continue

            if line[0] == '(':
                fixed_line = '[' + line.strip().replace(') (', '), (') + ']'
                params = ast.literal_eval(fixed_line)
                nested = [list(zip(key.split(','), np.array(val).flatten())) for (key, val) in params]
                unnested = [item for sublist in nested for item in sublist]
                grid_search_results[-1].append({key: val for (key, val) in unnested})
                grid_search_results[-1].append([])
                continue

            result = line

            if result[0] == '[':
                grid_search_results[-1][-1].append(ast.literal_eval(result))
                continue

    grid_search_results = [(params, np.array(values)) for params, values in grid_search_results]
    return grid_search_results

In [121]:
DIR_PATH = 'blueberry/grid_search_results'
res_files = [os.path.join(DIR_PATH, f) for f in os.listdir(DIR_PATH) if os.path.isfile(os.path.join(DIR_PATH, f))]

In [122]:
all_res = [parse_results(f) for f in res_files]
all_res = [item for row in all_res for item in row]

In [123]:
all_params = [p for p, _ in all_res]
all_values = [np.array(v) for _, v in all_res]

In [124]:
hyp_indices = [i for i, p in enumerate(all_params) if p['model_type'] == 'hyp']
euc_indices = [i for i, p in enumerate(all_params) if p['model_type'] == 'euc']

In [125]:
hyp_params = []
hyp_values = []

euc_params = []
euc_values = []

In [126]:
for i, (p, v) in enumerate(zip(all_params, all_values)):
    if i in hyp_indices:
        hyp_params.append(p)
        hyp_values.append(v)
    else:
        euc_params.append(p)
        euc_values.append(v)

In [127]:
hyp_size_values = {size: [] for size in set([p['layer_size'] for p in hyp_params])}
euc_size_values = {size: [] for size in set([p['layer_size'] for p in euc_params])}

In [128]:
for p, v in zip(hyp_params, hyp_values):
    hyp_size_values[p['layer_size']].append(v.mean(axis=0))
for p, v in zip(euc_params, euc_values):
    euc_size_values[p['layer_size']].append(v.mean(axis=0))

In [129]:
for size in [2,4,8,16,32,64,128,256]:
    # print(f'{size}: {min([v.min() for v in hyp_size_values[size]]):.3}, {min([v[-1] for v in hyp_size_values[size]]):.3}')
    # print(f'{size}: {min([v[-1] for v in hyp_size_values[size]]):.2}')
    print(f'{size}: {min([min(min(v[:100][::10]), min(v[::100]), v[-1]) for v in hyp_size_values[size]]):.2}')

2: 0.033
4: 0.029
8: 0.031
16: 0.029
32: 0.031
64: 0.03
128: 0.037
256: 0.036


In [130]:
# for size in sorted(euc_size_values.keys()):
for size in [2,4,8,16,32,64,128,256]:
    # print(f'{size}: {min([v.min() for v in euc_size_values[size]]):.3}, {min([v[-1] for v in euc_size_values[size]]):.3}')
    print(f'{size}: {min([min(min(v[:100][::10]), min(v[::100]), v[-1]) for v in euc_size_values[size]]):.2}')

2: 0.029
4: 0.029
8: 0.028
16: 0.029
32: 0.028
64: 0.029
128: 0.029
256: 0.029


In [83]:
[(i, v.mean(axis=0)) for i, v in enumerate(hyp_values)]

[(0,
  array([0.61252855, 0.36514554, 0.16026333, 0.17057315, 0.23422714,
         0.26036439, 0.24781374, 0.21292102, 0.16959285, 0.13101625,
         0.10737123, 0.09465594, 0.08836947, 0.08375714, 0.07860323,
         0.0727311 , 0.06705256, 0.06163551, 0.05766595, 0.05508973,
         0.05305634, 0.05161343, 0.0508868 , 0.0532942 , 0.0563193 ,
         0.05872158, 0.06040115, 0.06141269, 0.06144105, 0.06058   ,
         0.05904402, 0.05700416, 0.0545166 , 0.05163211, 0.04877203,
         0.04700182, 0.04574053, 0.04523969, 0.04521677, 0.04547105,
         0.04563281, 0.04574959, 0.04575107, 0.04569324, 0.04554252,
         0.04533425, 0.04507864, 0.04479833, 0.04452392, 0.04427906,
         0.04423034, 0.04423728, 0.04434978, 0.04443781, 0.04446871,
         0.04446462, 0.04439545, 0.04426778, 0.04408938, 0.04390522,
         0.04373577, 0.0436465 , 0.04365865, 0.04369356, 0.04372432,
         0.04374849, 0.0437746 , 0.04381555, 0.04386442, 0.04389748,
         0.04391957, 0.043934

In [93]:
hyp_params[49]

{'model_type': 'hyp',
 'num_hidden_layers': 32,
 'layer_size': 64,
 'lr': 0.01,
 'weight_decay': 0.003,
 'batch_size': 1024,
 'epochs': 100,
 'curvature': -1}